In [1]:
import psycopg2
import pandas as pd

Connection with DB

In [2]:
import psycopg2

connect = psycopg2.connect(
    host = "localhost",
    user = "root",
    password = "root",
    database = "postgres",
    port = "5432"

)
connect.autocommit = True

SQL query to create the table

In [3]:
def makeQuery():
    cursor = connect.cursor()

    # SQL query to create the table
    query = """
    CREATE TABLE IF NOT EXISTS applicants (
        first_name VARCHAR(500),
        last_name VARCHAR(500),
        email VARCHAR(500),
        application_date VARCHAR(500),
        country VARCHAR(500),
        years_of_experience VARCHAR(500),
        seniority VARCHAR(500),
        technology VARCHAR(500),
        code_challenge_score VARCHAR(500),
        technical_interview_score VARCHAR(500)
    );
    """

    cursor.execute(query)
    cursor.close()

makeQuery()

# Insert the data from candidates.csv uploaded in to Pgadmin.

query = """
copy public.applicants (first_name, 
last_name, email, application_date, 
country, years_of_experience, seniority, 
technology, code_challenge_score, technical_interview_score) 
FROM '<STORAGE_DIR>/candidates.csv' DELIMITER ';' CSV QUOTE '\"' ESCAPE '''';""}
"""
makeQuery()

EDA

Make the pandas Dataframe

In [4]:
cursor = connect.cursor()
cursor.execute("SELECT * FROM applicants")
dbtable = cursor.fetchall()
df = pd.DataFrame(dbtable)

#1 Data Understanding:
Displays the first rows of data, displays information about the DataFrame, and displays statistical summary of the DataFrame

In [5]:
print(df.head())  # Muestra las primeras filas de datos
print(df.info())  # Muestra información sobre el DataFrame
print(df.describe())  # Resumen estadístico del DataFrame
print(df.dtypes) # Tipos de dato por columnas
df.isna().sum() # Suma de nulos por columnas

            0          1                         2                 3        4  \
0  First Name  Last Name                     Email  Application Date  Country   
1  Bernadette  Langworth       leonard91@yahoo.com        2021-02-26   Norway   
2      Camryn   Reynolds       zelda56@hotmail.com        2021-09-09   Panama   
3       Larue     Spinka  okey_schultz41@gmail.com        2020-04-14  Belarus   
4        Arch     Spinka    elvera_kulas@yahoo.com        2020-10-01  Eritrea   

     5          6               7                     8  \
0  YOE  Seniority      Technology  Code Challenge Score   
1    2     Intern   Data Engineer                     3   
2   10     Intern   Data Engineer                     2   
3    4  Mid-Level  Client Success                    10   
4   25    Trainee       QA Manual                     7   

                           9  
0  Technical Interview Score  
1                          3  
2                         10  
3                          9  
4  

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
dtype: int64

#2 Data preparation:
First of all, change the hearders to the original names, not numbers.

In [6]:
# Extract the second row as column headers
new_headers = df.iloc[0]

# Assign the new headers to the DataFrame
df.columns = new_headers

# Drop the second row
df = df.drop(df.index[0])

print(df.head())

0  First Name   Last Name                      Email Application Date  \
1  Bernadette   Langworth        leonard91@yahoo.com       2021-02-26   
2      Camryn    Reynolds        zelda56@hotmail.com       2021-09-09   
3       Larue      Spinka   okey_schultz41@gmail.com       2020-04-14   
4        Arch      Spinka     elvera_kulas@yahoo.com       2020-10-01   
5       Larue  Altenwerth  minnie.gislason@gmail.com       2020-05-20   

0  Country YOE  Seniority                         Technology  \
1   Norway   2     Intern                      Data Engineer   
2   Panama  10     Intern                      Data Engineer   
3  Belarus   4  Mid-Level                     Client Success   
4  Eritrea  25    Trainee                          QA Manual   
5  Myanmar  13  Mid-Level  Social Media Community Management   

0 Code Challenge Score Technical Interview Score  
1                    3                         3  
2                    2                        10  
3                   10 

I change the data type of some columns that are numeric and were as object

In [7]:
df['YOE'] = df['YOE'].astype(int)
df['Code Challenge Score'] = df['Code Challenge Score'].astype(int)
df['Technical Interview Score'] = df['Technical Interview Score'].astype(int)
df['Application Date'] = pd.to_datetime(df['Application Date'])
print(df.dtypes)

0
First Name                           object
Last Name                            object
Email                                object
Application Date             datetime64[ns]
Country                              object
YOE                                   int32
Seniority                            object
Technology                           object
Code Challenge Score                  int32
Technical Interview Score             int32
dtype: object


I divide the Application Date column into three other new columns: Year, month and day.

In [8]:
df['year'] = df['Application Date'].dt.year
df['month'] = df['Application Date'].dt.month
df['day'] = df['Application Date'].dt.day
print(df['year'], df['month'], df['day'])

1        2021
2        2021
3        2020
4        2020
5        2020
         ... 
49996    2022
49997    2020
49998    2018
49999    2020
50000    2022
Name: year, Length: 50000, dtype: int64 1         2
2         9
3         4
4        10
5         5
         ..
49996     1
49997     6
49998    12
49999     5
50000     6
Name: month, Length: 50000, dtype: int64 1        26
2         9
3        14
4         1
5        20
         ..
49996     9
49997     2
49998    15
49999    30
50000    13
Name: day, Length: 50000, dtype: int64


https://github.com/Davele12/workshop1